In [1]:
# install all required packages for the project using requirements.txt

# !pip install -r src/requirements.txt

Once all reuired dependencies are installed, set up aws cli and configure it with your aws account credentials.


In [66]:
# import all required modules

import boto3
import json
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.memory import (
    ConversationSummaryBufferMemory,
    CombinedMemory,
    ConversationBufferWindowMemory,
)
from langchain.prompts import PromptTemplate

In [2]:
# Create a session using the credentials from the .credentials file
session = boto3.Session()

# Get the service resource.
dynamodb = session.resource("dynamodb", region_name="us-east-1")

# Create a client for the Lambda service

lambda_client = session.client("lambda", region_name="us-east-1")

In [9]:
# Create SessionTable and SessionSummaryTable to Store User Chats and Summaries of Chats respectively.

# create only if not exists

try:
    session_table = dynamodb.create_table(
        TableName="SessionTable",
        KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
        AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
        BillingMode="PAY_PER_REQUEST",
    )
except Exception as e:
    # print(e)
    print("Session Table already exists")

Session Table already exists


In [6]:
try:
    summary_table = dynamodb.create_table(
        TableName="SessionSummaryTable",
        KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
        AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
        BillingMode="PAY_PER_REQUEST",
    )
except Exception as e:
    # print(e)
    print("SessionSummaryTable already exists")

SessionSummaryTable already exists


In [11]:
session_table = dynamodb.Table("SessionTable")
summary_table = dynamodb.Table("SessionSummaryTable")

In [12]:
# wait until both the tables are created

session_table.meta.client.get_waiter("table_exists").wait(TableName="SessionTable")
summary_table.meta.client.get_waiter("table_exists").wait(
    TableName="SessionSummaryTable"
)

In [13]:
# create a LLM

llm = ChatOpenAI(temperature=0)

In [77]:
# define constants

# session id. This is the id of the user who is chatting with the bot.
session_id = "3"

# define how many recent conversations to be sent to the OPENAI API

num_recent_conversations = 10

# define token limit for the summary.

summary_token_limit = 100

In [87]:
# create a memory object that pulls latest num_recent_conversations from the dynamodb table

message_history = DynamoDBChatMessageHistory(
    table_name="SessionTable", session_id=session_id
)

conv_memory = ConversationBufferWindowMemory(
    chat_memory=message_history,
    k=num_recent_conversations,
    input_key="input",
    memory_key="chat_history_lines",
)

In [88]:
# create another memory object that is initialized with the existing summary of the conversation

response = summary_table.get_item(Key={"SessionId": session_id})

if response and "Item" in response:
    summary = response["Item"]["Summary"]
else:
    summary = ""
summary_memory = ConversationSummaryBufferMemory(
    llm=ChatOpenAI(temperature=0),
    memory_key="summary",
    moving_summary_buffer=summary,
    max_token_limit=summary_token_limit,
    input_key="input",
)

In [89]:
# create a combined memory object that combines the above two memories

memory = CombinedMemory(memories=[conv_memory, summary_memory])

In [92]:
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation:
{summary}
Current conversation:
{chat_history_lines}
Human: {input}
AI:"""

prompt = PromptTemplate(
    input_variables=["summary", "chat_history_lines", "input"],
    template=_DEFAULT_TEMPLATE,
)

# create a conversation chain object

conversation = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=prompt)

In [93]:
memory.load_memory_variables({})

{'chat_history_lines': '', 'summary': ''}

In [94]:
conversation.predict(input="Hello")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation:

Current conversation:

Human: Hello
AI:

> Finished chain.


'Hello there! How can I assist you today?'

In [101]:
answer = conversation.predict(input="Hello, my name is Adam.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation:
System: Adam introduces himself to the AI and the AI offers assistance. The AI asks if there is anything specific Adam needs help with and Adam responds that he needs help with his taxes.
AI: Of course, Adam! I can definitely help you with that. Do you have any specific questions or concerns about your taxes? It's important to note that I am not a licensed tax professional, but I can provide general information and guidance based on my knowledge and resources.
Human: Hello, my name is Adam.
AI: Hello again, Adam! Is there anything specific you need help with today?
Current conversation:
Human: Hello
AI: Hello there! How can I assist you today?
Human: Hello, my name 

In [103]:
type(answer)

str

In [96]:
conversation.predict(input="I'm looking for help with my taxes.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation:
Human: Hello
AI: Hello there! How can I assist you today?
Human: Hello, my name is Adam.
AI: Nice to meet you, Adam! Is there anything specific you would like me to help you with?
Current conversation:
Human: Hello
AI: Hello there! How can I assist you today?
Human: Hello, my name is Adam.
AI: Nice to meet you, Adam! Is there anything specific you would like me to help you with?
Human: I'm looking for help with my taxes.
AI:

> Finished chain.


"Of course, Adam! I can definitely help you with that. Do you have any specific questions or concerns about your taxes? It's important to note that I am not a licensed tax professional, but I can provide general information and guidance based on my knowledge and resources."

In [86]:
session_table.delete_item(Key={"SessionId": session_id})

{'ResponseMetadata': {'RequestId': 'JVSOHFAQBQNJ6SC5BKM0TPD0VJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 21 Jun 2023 18:19:32 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'JVSOHFAQBQNJ6SC5BKM0TPD0VJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [98]:
conv_memory.chat_memory.messages

[HumanMessage(content='Hello', additional_kwargs={}, example=False),
 AIMessage(content='Hello there! How can I assist you today?', additional_kwargs={}, example=False),
 HumanMessage(content='Hello, my name is Adam.', additional_kwargs={}, example=False),
 AIMessage(content='Nice to meet you, Adam! Is there anything specific you would like me to help you with?', additional_kwargs={}, example=False),
 HumanMessage(content="I'm looking for help with my taxes.", additional_kwargs={}, example=False),
 AIMessage(content="Of course, Adam! I can definitely help you with that. Do you have any specific questions or concerns about your taxes? It's important to note that I am not a licensed tax professional, but I can provide general information and guidance based on my knowledge and resources.", additional_kwargs={}, example=False)]

In [104]:
summary_memory.moving_summary_buffer

'Adam introduces himself to the AI and the AI offers assistance with his taxes. The AI asks if there is anything specific Adam needs help with and offers general information and guidance based on its knowledge and resources, but notes that it is not a licensed tax professional.'

In [105]:
summary_table.delete_item(Key={"SessionId": "10"})
session_table.delete_item(Key={"SessionId": "10"})

{'ResponseMetadata': {'RequestId': '1FNQSL0TLBTB5VU55UI1VT2DSJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 21 Jun 2023 20:08:28 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1FNQSL0TLBTB5VU55UI1VT2DSJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}